In [5]:
import os

In [6]:
os.chdir('../')

In [7]:
%pwd

'/Users/ravina/Desktop/project_mlflow'

In [49]:
MLFLOW_TRACKING_URI = 'https://dagshub.com/ravina029/mlflow_project.mlflow'
MLFLOW_TRACKING_USERNAME = 'ravina029'
MLFLOW_TRACKING_PASSWORD='221d4e3a527ff8b9aef06e059d7efc4e89963e11'


In [50]:
os.environ[MLFLOW_TRACKING_URI]='https://dagshub.com/ravina029/mlflow_project.mlflow'
os.environ[MLFLOW_TRACKING_USERNAME]='ravina029'
os.environ[MLFLOW_TRACKING_PASSWORD]='221d4e3a527ff8b9aef06e059d7efc4e89963e11'

In [51]:
from dataclasses import dataclass 
from pathlib import Path

@dataclass(frozen=True)  #this is not python class but dataclass, here you can define the veriables without using self keyword.
class ModelEvaluationConfig:
    root_dir:Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name:Path
    target_column: str
    mlflow_uri: str
    

In [52]:
from mlproject.constants import *
from mlproject.utils.common import read_yaml,create_directories

In [58]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    

    def get_model_evaluation_config(self)-> ModelEvaluationConfig:
        config=self.config.model_trainer
        params=self.params.ElasticNet
        schema=self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config=ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri= "https://dagshub.com/ravina029/mlflow_project.mlflow"
            )

        return model_evaluation_config

In [59]:
import urllib.request  

In [60]:
#components

import os
import pandas as pd
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [61]:
class ModelEvaluation:
    def __init__(self,config:ModelEvaluationConfig):
        self.config=config

    def eval_metrics(self,actual,pred):
            rmse=np.sqrt(mean_squared_error(actual,pred))
            mae=mean_absolute_error(actual,pred)
            r2=r2_score(actual,pred)
            return rmse,mae,r2
        

    def log_into_mlflow(self):
            test_data=pd.read_csv(self.config.test_data_path)
            model=joblib.load(self.config.model_path)

            test_x=test_data.drop([self.config.target_column],axis=1)
            test_y=test_data[[self.config.target_column]]

            mlflow.set_registry_uri(self.config.mlflow_uri)
            tracking_url_type_score=urlparse(mlflow.get_tracking_uri()).scheme

            with mlflow.start_run():
                predicted_qualities=model.predict(test_x)
                (rmse,mae,r2)=self.eval_metrics(test_y,predicted_qualities)

                #saving metrics as local
                scores={"rmse":rmse,"mae":mae,"r2":r2}
                save_json(path=Path(self.config.metric_file_name), data=scores)

                mlflow.log_param(self.config.all_params)

                mlflow.log_metric("rmse",rmse)
                mlflow.log_metric("r2",r2)
                mlflow.log_metric("mae",mae)

                #Model registry doesn't work with file store
                if tracking_url_type_score!='file':
                    #Regitster the model
                    #There are other ways to use the Model registry, which depends on the use case,
                    #please refer to the doc for more information:
                    #https://mlflow.org/docs/latest/models.html
                    mlflow.sklearn.load_model(model,"model", registered_model_name='ElasticnetModel')
                else:
                    mlflow.sklearn.log_model(model,'model')    
                    







In [62]:
try: 
    config=ConfigurationManager()
    model_evaluation_config=config.get_model_evaluation_config()
    model_evaluation_config=ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2024-01-12 13:03:53,940:INFO:yaml file:config/config.yaml loaded successfully]
[2024-01-12 13:03:53,942:INFO:yaml file:params.yaml loaded successfully]
[2024-01-12 13:03:53,944:INFO:yaml file:Schema.yaml loaded successfully]
[2024-01-12 13:03:53,945:INFO:created directory at:artifacts]
[2024-01-12 13:03:53,947:INFO:created directory at:artifacts/model_trainer]


BoxKeyError: "'ConfigBox' object has no attribute 'model_path'"